In [1]:
import os
import openai
import pandas as pd
import tiktoken
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

#### Necessary pip installation commands include:
- pip install openai
- pip install pandas
- pip install tiktoken
- pip install numpy
- pip install python-dotenv
- pip install langchain
- pip install langchain-pinecone
- pip install langchain-core
- pip install langchain-openai

#### <b>Note</b>: have a <b>.env</b> file already created for accessing API key

In [2]:
# retrieving API Key from OpenAI platform
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv('FINE_TUNED_TOXIC_DETECTION_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index = os.getenv('PINECONE_GPT')

In [3]:
# initializing Pinecone vector database instance
docSearch = Pinecone(
    index_name=pc_index,
    embedding=OpenAIEmbeddings()
)
# instantiating client with API key
fine_tuned = OpenAI(
    api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY
)

C:\Users\elija\AppData\Local\Temp\ipykernel_20320\2326318028.py:2: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 0.3.0. Use :class:`~PineconeVectorStore` instead.
  docSearch = Pinecone(


In [4]:
# appends one/few shot examples to evaluation prompt
def shot_additions(examples):
   evaluation_prompt = ''
   # Read the evaluation prompt from the text file with utf-8 encoding
   with open("../../data/text/rag_prompts/gpt-4-two-shot-prompt.txt", "r", encoding="utf-8") as file:
        evaluation_prompt += file.read()
   print(f'Examples: {len(examples)}')
   for example in examples:
       # separates example key and values
       split_ex = example.split(' - ')
       comment = split_ex[0]
       label = split_ex[1]
       evaluation_prompt += '\n\nText: \"' + comment + "\"" + '\n\n' + label

   evaluation_prompt += '\n\nClassify the following comment:'

   return evaluation_prompt

In [5]:
# queries vector database for custom,
# with similar examples to user prompt
def rag_prompt(text):
    two_shots = []
    # queries Pinecone database
    search_results = docSearch.max_marginal_relevance_search(text, k=25, fetch_k=50)
    print(f'Search Results Length: {len(search_results)}')
    for i in range(len(search_results)):
        content = search_results[i].page_content
        # prevents repetition which will cause errors within OpenAI
        if i != 250:
            two_shots.append(content)
        # two valid examples found
        if len(two_shots) == 2:
            print(f'Two Shots Found!')
            break
    return shot_additions(two_shots)

In [6]:
# tests accuracy of chosen model against unique prompt and data
def accuracy_testing(data, evaluation_prompt, model, use_rag = False):
    # chooses columns of focus
    tuned = pd.DataFrame(columns=['Text', 'Toxic'])
    for index, row in data.iterrows():
        text = row['Text']
        print(f"Index: {index}")
        completion = fine_tuned.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system", 
                    "content": evaluation_prompt if use_rag == False else rag_prompt(text)
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        tuned.loc[index] = [text, completion.choices[0].message.content]
    # comparing results of model to dataset
    compare = tuned['Toxic'] == data['Toxic']
    accuracy = compare.values.sum() / compare.size
    return accuracy

In [7]:
# retrieving datasets utilized for evaluation
sugarai = pd.read_csv('../../data/csv/sugar_ai_toxicity_evaluation_set.csv')
unintended_bias = pd.read_csv('../../data/csv/unintended_bias_toxicity_classification_set.csv')
sugarai['Toxic'] = sugarai['Toxic'].apply(lambda x: str(int(x)))
unintended_bias['Toxic'] = unintended_bias['Toxic'].apply(lambda x: str(int(x)))

In [8]:
evaluation_prompt = ''
# Read the evaluation prompt from the text file with utf-8 encoding
with open("../../data/text/fine_tuned_prompts/gpt-4-two-shot-prompt.txt", "r", encoding="utf-8") as file:
    evaluation_prompt += file.read()

In [10]:
# testing accuracy of Sugar AI Toxicity Classification dataset on original model
sugarai_original_test_accuracy = accuracy_testing(sugarai, "Detect whether either is 1 for toxic or 0 for non-toxic", "gpt-4o")
print(f"Accuracy: {sugarai_original_test_accuracy * 100:.2f}%")

Accuracy: 90.98%


In [11]:
# testing accuracy of Sugar AI Toxicity Classification dataset on fine-tuned model
sugarai_fine_tuned_test_accuracy = accuracy_testing(sugarai, evaluation_prompt, "ft:gpt-4o-2024-08-06:personal::ASwKLqOH")
print(f"Accuracy: {sugarai_fine_tuned_test_accuracy * 100:.2f}%")

Accuracy: 93.69%


In [9]:
# testing accuracy of Unintended Sugar AI Toxicity Classification dataset on fine-tuned model with RAG
sugarai_rag_tuned_test_accuracy = accuracy_testing(sugarai, evaluation_prompt, "ft:gpt-4o-2024-08-06:personal::ASwKLqOH", True)
print(f"Accuracy: {sugarai_rag_tuned_test_accuracy * 100:.2f}%")

In [12]:
# testing accuracy of Unintended Bias Toxicity Classification dataset on original model
unintended_bias_original_test_accuracy = accuracy_testing(unintended_bias, "Detect whether either is 1 for toxic or 0 for non-toxic", "gpt-4o")
print(f"Accuracy: {unintended_bias_original_test_accuracy * 100:.2f}%")

Accuracy: 69.50%


In [13]:
# testing accuracy of Unintended Bias Toxicity Classification dataset on fine-tuned model
unintended_bias_fine_tuned_test_accuracy = accuracy_testing(unintended_bias, evaluation_prompt, "ft:gpt-4o-2024-08-06:personal::ASwKLqOH")
print(f"Accuracy: {unintended_bias_fine_tuned_test_accuracy * 100:.2f}%")

Accuracy: 80.20%


In [ ]:
# testing accuracy of Unintended Bias Toxicity Classification dataset on fine-tuned model with RAG
unintended_bias_rag_tuned_test_accuracy = accuracy_testing(unintended_bias, evaluation_prompt, "ft:gpt-4o-2024-08-06:personal::ASwKLqOH", True)
print(f"Accuracy: {unintended_bias_rag_tuned_test_accuracy * 100:.2f}%")

In [13]:
# retrieving dataset utilized for evaluation
tuned = pd.DataFrame(columns=['Text', 'Toxic'])

In [14]:
len(unintended_bias)

1000

In [ ]:
# processing fine-tuned GPT-4o model across entire dataset 
# with advanced prompting with RAG support
# for index, row in unintended_bias.iterrows():
#     text = row['Text']
#     print(f"Index: {index}")
#     completion = fine_tuned.chat.completions.create(
#         model="ft:gpt-4o-2024-08-06:personal::ASwKLqOH",
#         messages=[
#             {
#                 "role": "system", 
#                 "content": rag_prompt(text)
#             },
#             {
#                 "role": "user",
#                 "content": text
#             }
#         ]
#     )
#     tuned.loc[index] = [row['Text'], completion.choices[0].message.content]

In [17]:
# comparing results of model to dataset
compare = tuned['Toxic'] == unintended_bias['Toxic']
gptfour_finetuned_accuracy = compare.values.sum() / compare.size

In [18]:
# fine-tuned GPT-4o model accuracy with advanced prompted engineering
# (role prompting, two-shot examples) and RAG for few-shot support
print(f"Accuracy: {gptfour_finetuned_accuracy * 100:.2f}%")

Accuracy: 88.40%
